<a href="https://colab.research.google.com/github/Ananassio/Data-Science_lab/blob/main/week_6/6_b_AutoSkLearn_Regression_NY_Taxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
#in colab, we need to clone the data from the repo
!git clone https://github.com/keuperj/DATA.git

Cloning into 'DATA'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 96 (delta 1), reused 9 (delta 1), pack-reused 87
Unpacking objects: 100% (96/96), done.
Checking out files: 100% (66/66), done.


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv('DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [4]:
# install 
!pip install scikit-optimize

     |████████████████████████████████| 100 kB 4.4 MB/s 


### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [5]:
from sklearn.metrics import accuracy_score

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70) #split 85% train 15% test

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [8]:
#------------------------------------------------------------------------------------------
# Linear Regression
%%time
LR = LinearRegression(n_jobs=-1)
LR.fit(X_train,y_train)
print(LR.score(X_test,y_test))

0.7353412380308091
CPU times: user 237 ms, sys: 81.9 ms, total: 319 ms
Wall time: 227 ms


In [9]:
#------------------------------------------------------------------------------------------
# Randomforest Regression
%%time
RFR =RandomForestRegressor(n_jobs=-1, n_estimators=100)
RFR.fit(X_train,y_train)
print(RFR.score(X_test,y_test))

0.8081623700903032
CPU times: user 3min 55s, sys: 1.12 s, total: 3min 56s
Wall time: 2min 14s


In [10]:
#------------------------------------------------------------------------------------------
# SVM Regression
SVM = make_pipeline(StandardScaler(), SVR(max_iter=50))
SVM.fit(X_train, y_train)
print(SVM.score(X_test,y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


-15.692103780352493


In [11]:
#------------------------------------------------------------------------------------------
# optimazation Randomforest Regression
serches_RFR = {'n_estimators':[100,250,500],
               'max_depth': [1,5,10]
               }

In [12]:
%%time
opt_RFR = BayesSearchCV(RFR,search_spaces=serches_RFR,n_iter=32,random_state=0,n_jobs=-1)
# executes bayesian optimization
opt_RFR.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(opt_RFR.score(X_test, y_test))

/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.7/dist-packa

0.8090575311325563
CPU times: user 11min 7s, sys: 23.8 s, total: 11min 31s
Wall time: 4h 28min 32s


In [16]:
#------------------------------------------------------------------------------------------
# optimazation SVM Regression
serches_SVM = {'svr__C':Real(1e-5, 1e+6, prior='log-uniform'),
               'svr__gamma': Real(1e-5, 1e+3, prior='log-uniform')
               }

In [14]:
SVM.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'svr', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'svr__C', 'svr__cache_size', 'svr__coef0', 'svr__degree', 'svr__epsilon', 'svr__gamma', 'svr__kernel', 'svr__max_iter', 'svr__shrinking', 'svr__tol', 'svr__verbose'])

In [17]:
%%time
opt_SVM = BayesSearchCV(SVM,search_spaces=serches_SVM,n_iter=32,random_state=0,n_jobs=-1)
# executes bayesian optimization
opt_SVM.fit(X_train, y_train)

# model can be saved, used for predictions or scoring
print(opt_SVM.score(X_test, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


-9.30610787454622
CPU times: user 41.6 s, sys: 20.3 s, total: 1min 1s
Wall time: 3min 31s
